# Administration: Mirror groups of user

New users are not part of any group. This scripts adds a new user "NEWUSER" to the same groups as
another (existing) user "TEMPLATEUSER"

### Static configuration variables

In [ ]:
osparcURL = 'https://' + os.environ.get('MACHINE_FQDN')
newUserMail = "kaiser@itis.swiss"
templateUserMail = "zhuang@itis.swiss"

## Clone oSparc Repo and install python dependencies

In [ ]:
print("FYI: You can ignore even fatal errors in this section....")
!git clone https://github.com/ITISFoundation/osparc-simcore.git
!python -m pip install sqlalchemy
!python -m pip install psycopg2-binary

## Fetching Information

### Fetching Information: pgSQL Database - Get userID of templateUser

In [ ]:
import sqlalchemy as db
import psycopg2
import os
import json
import copy
#######################
PG_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
PG_ENDPOINT=os.environ.get('POSTGRES_ENDPOINT')
PG_DB=os.environ.get('POSTGRES_DB')
PG_USER=os.environ.get('POSTGRES_USER')
pgEngineURL= "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        user=PG_USER,
        password=PG_PASSWORD,
        database=PG_DB,
        host=PG_ENDPOINT.split(":")[0],
        port=int(PG_ENDPOINT.split(":")[1]),
    )
engine = db.create_engine(pgEngineURL)
connection = engine.connect()
metadata = db.MetaData()
users = db.Table('users', metadata, autoload=True, autoload_with=engine)
##########
# Automatically building datestamp iterator for current invite:
query = db.select([users]).where(users.c.email.like("%" + templateUserMail + "%"))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
print("UserID of template user: ",ResultSet[0][0])
templateUserID = copy.deepcopy(ResultSet[0][0])
query = db.select([users]).where(users.c.email.like("%" + newUserMail + "%"))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
print("UserID of new user: ",ResultSet[0][0])
newUserID = copy.deepcopy(ResultSet[0][0])

In [ ]:
user2groups = db.Table('user_to_groups', metadata, autoload=True, autoload_with=engine)
queryNewUser = db.select([user2groups]).where(user2groups.c.uid == newUserID)
ResultProxy = connection.execute(queryNewUser)
ResultSetUser2Groups = ResultProxy.fetchall()
listGroupsAlreadyMember = [q[1] for q in ResultSetUser2Groups]
queryTemplateUser = db.select([user2groups]).where(user2groups.c.uid == templateUserID)
ResultProxy = connection.execute(queryTemplateUser)
ResultSetUser2Groups = ResultProxy.fetchall()
#####
groupIDsToBeAdded = []
for i, group in enumerate(ResultSetUser2Groups):
    groupsTable = db.Table('groups', metadata, autoload=True, autoload_with=engine)
    queryGroups = db.select([groupsTable]).where(db.and_(groupsTable.c.gid == group[1], groupsTable.c.type != "PRIMARY", groupsTable.c.type != "EVERYONE"))
    ResultProxyGroups = connection.execute(queryGroups)
    ResultSetGroups = ResultProxyGroups.fetchall()
    if len(ResultSetGroups) == 1:
        if ResultSetGroups[0][0] not in listGroupsAlreadyMember:
            print(ResultSetGroups[0][:3])
            groupIDsToBeAdded.append(group)
        else:
            print(ResultSetGroups[0][:3], " - User is already member, skipping...")

### ---> Please confirm that you want to add the NEWUSER to the groups printed above before procedeeding

## Modifying Database: Add user to groups

In [ ]:
print("User with ID ",newUserID, " will be added to ", len(groupIDsToBeAdded), " groups in total.")

In [ ]:
#Inserting new record into database
import datetime
dateTimeObj = datetime.datetime.now()
for i in groupIDsToBeAdded:
    print("Adding user ", newUserID, "to group ", i[1], " with rights ", i[4])
    query = db.insert(user2groups).values(uid=newUserID, gid=i[1], created=dateTimeObj, modified=dateTimeObj,access_rights=i[4]) 
    ResultProxy = connection.execute(query)